In [1]:
import warnings
import pandas as pd
import scanpy as sc
import numpy as np
import scrublet as scr
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Construct AnnData object from sparse matrix, cell barcode list, and gene list

The AnnData object is a widely used data structure in single-cell genomics analysis, provided by the Python package anndata. It serves as a container for high-dimensional single-cell data, such as gene expression from sc/snRNA-seq. The AnnData object encapsulates the data matrix along with various annotations and metadata associated with the cells and features (genes). Here's an overview of the structure of an AnnData object:

**Data Matrix**: The primary component of an AnnData object is a two-dimensional data matrix. It represents the expression values of genes (features) across individual cells. By convention, the matrix is stored as a dense or sparse matrix object, depending on the size and sparsity of the data. The matrix typically has cells as rows and genes as columns.

**Observations**: Each row of the data matrix corresponds to an individual cell (aka observation). The AnnData object stores various properties or annotations associated with each cell, for example the cell's barcode, the type of the cell, or sample information associated with the sample the cell came from. These annotations are typically stored as a pandas DataFrame, where each row corresponds to a cell and each column represents a specific annotation.

**Variables**: Each column of the data matrix corresponds to a gene (aka variable). The AnnData object also stores various properties or annotations associated with each feature. For example, it can include gene names, gene IDs, and gene type. Similar to cell annotations, feature annotations are stored as a pandas DataFrame, where each row corresponds to a gene and each column represents a specific annotation.

See [anndata documentation](https://anndata.readthedocs.io/en/latest/) for more details.


In [2]:
# the directory containing the data
path = "/data/class/cosmos2023/PUBLIC/terminator/GSE156498/"

# Sample 1

In [3]:
sample = 'GSM4732631_WT'

In [4]:
# Load sparse matrix from file
mtx = sc.read_mtx(path + sample + '_matrix.mtx').T

In [5]:
# Load gene IDs or names from CSV file
var = pd.read_csv(path + sample + '_features.tsv', sep = '\t', header=None)

# Set new column names
var.columns = ['gene_id', 'gene_name', 'assay']

# Load observations from CSV file
obs = pd.read_csv(path + sample + '_barcodes.tsv', sep = '\t', header=None)
obs.columns = ['raw_bc']
obs['bc'] = obs['raw_bc'] + '_' + sample
obs['sample'] = sample

In [6]:
mtx.obs = obs
mtx.var = var

In [7]:
adata1 = ad.AnnData(X=mtx.X, var=var, obs=obs)

/opt/apps/python/3.8.0/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [8]:
adata1.obs.index = adata1.obs['bc']

In [9]:
adata1.var_names = adata1.var['gene_name'] 
adata1.var_names_make_unique()

# Sample 2

In [10]:
sample = 'GSM4732632_D51'

In [11]:
# Load sparse matrix from file
mtx = sc.read_mtx(path + sample + '_matrix.mtx').T

In [12]:
# Load gene IDs or names from CSV file
var = pd.read_csv(path + sample + '_features.tsv', sep = '\t', header=None)

# Set new column names
var.columns = ['gene_id', 'gene_name', 'assay']

# Load observations from CSV file
obs = pd.read_csv(path + sample + '_barcodes.tsv', sep = '\t', header=None)
obs.columns = ['raw_bc']
obs['bc'] = obs['raw_bc'] + '_' + sample
obs['sample'] = sample

In [13]:
mtx.obs = obs
mtx.var = var

In [14]:
adata2 = ad.AnnData(X=mtx.X, var=var, obs=obs)

/opt/apps/python/3.8.0/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [15]:
adata2.obs.index = adata2.obs['bc']

In [16]:
adata2.var_names = adata2.var['gene_name'] 
adata2.var_names_make_unique()

In [17]:
adata2

AnnData object with n_obs × n_vars = 4486 × 31053
    obs: 'raw_bc', 'bc', 'sample'
    var: 'gene_id', 'gene_name', 'assay'

# Merge samples into 1 object

In [18]:
adata = ad.concat([adata1, adata2], axis = 0)

In [19]:
adata

AnnData object with n_obs × n_vars = 11581 × 31053
    obs: 'raw_bc', 'bc', 'sample'

In [20]:
#sc.pp.filter_cells(adata, min_counts=500)

In [21]:
adata.write_h5ad("/data/class/cosmos2023/PUBLIC/terminator/scanpy/adata.h5ad")
